In [9]:
import joblib
import json
import os

import mlflow
import pandas as pd

In [16]:
# Loading the trained model
with open("model/fitted_model_cbr_l.pkl", "rb") as fd:
    model = joblib.load(fd)  # Эта строка внутри блока with (имеет отступ)

# Retrieving Catboost model parameters (без отступа, так как это уже вне блока with)
model_params = model["model"].get_params()

In [17]:
model_params

{'loss_function': 'RMSE',
 'random_seed': 42,
 'verbose': 100,
 'cat_features': [11, 12, 13]}

In [18]:
with open("metrics/eval_results_cbr_l.json") as json_file:
        metrics = json.load(json_file)

In [19]:
metrics

{'r2_score': 0.7007744928684252,
 'rmse': 2462549.744002088,
 'mae': 1873492.8414151592,
 'mape': 1.470389120444638}

In [2]:
# сделайте разделение изначального набора данных в этой ячейке
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import os
from autofeat import AutoFeatRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
#from category_encoders import CatBoostEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, KBinsDiscretizer
from sklearn.model_selection import StratifiedKFold, cross_validate
from catboost import CatBoostRegressor
import warnings

In [3]:
df = pd.read_csv('data/initial_data.csv')

In [4]:
df

,id,building_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,price
0,9018,17420,1999,4,55.647701,37.761356,2.64,460,14,True,9,10.3,46.599998,3,False,False,80.000000,13500000
1,9019,16317,1995,4,55.687134,37.863522,2.64,186,9,True,4,10.0,17.000000,1,False,False,45.000000,5720000
2,9020,23212,2015,4,55.697556,37.924061,2.74,320,17,True,5,8.0,19.000000,1,False,False,37.799999,6000000
3,9021,16949,1997,4,55.834301,37.352211,2.64,473,17,True,10,9.2,30.799999,2,False,False,55.000000,8750000
4,9022,10890,1974,4,55.815945,37.701008,2.64,143,9,True,7,7.0,33.000000,2,False,False,48.000000,8500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105107,141351,14149,1984,4,55.761795,37.410034,2.64,474,17,True,13,10.0,33.000000,2,False,False,57.700001,19300000
105108,141353,19460,2004,4,55.853230,37.646873,2.74,513,17,True,9,7.0,18.000000,1,False,False,38.000000,9200000
105109,141354,8225,1969,4,55.626068,37.608238,2.50,282,9,True,3,6.7,29.000000,2,False,False,45.000000,11300000
105110,141355,21093,2008,4,55.872646,37.634228,2.74,128,17,True,9,12.9,33.900002,2,False,False,64.000000,14800000


In [5]:
df = df.reset_index()

In [6]:
df

,index,id,building_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,price
0,0,9018,17420,1999,4,55.647701,37.761356,2.64,460,14,True,9,10.3,46.599998,3,False,False,80.000000,13500000
1,1,9019,16317,1995,4,55.687134,37.863522,2.64,186,9,True,4,10.0,17.000000,1,False,False,45.000000,5720000
2,2,9020,23212,2015,4,55.697556,37.924061,2.74,320,17,True,5,8.0,19.000000,1,False,False,37.799999,6000000
3,3,9021,16949,1997,4,55.834301,37.352211,2.64,473,17,True,10,9.2,30.799999,2,False,False,55.000000,8750000
4,4,9022,10890,1974,4,55.815945,37.701008,2.64,143,9,True,7,7.0,33.000000,2,False,False,48.000000,8500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105107,105107,141351,14149,1984,4,55.761795,37.410034,2.64,474,17,True,13,10.0,33.000000,2,False,False,57.700001,19300000
105108,105108,141353,19460,2004,4,55.853230,37.646873,2.74,513,17,True,9,7.0,18.000000,1,False,False,38.000000,9200000
105109,105109,141354,8225,1969,4,55.626068,37.608238,2.50,282,9,True,3,6.7,29.000000,2,False,False,45.000000,11300000
105110,105110,141355,21093,2008,4,55.872646,37.634228,2.74,128,17,True,9,12.9,33.900002,2,False,False,64.000000,14800000


In [9]:
df = df.drop(['id', 'building_id', 'studio'], axis=1)  

In [10]:
df

,index,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator,floor,kitchen_area,living_area,rooms,is_apartment,total_area,price
0,0,1999,4,55.647701,37.761356,2.64,460,14,True,9,10.3,46.599998,3,False,80.000000,13500000
1,1,1995,4,55.687134,37.863522,2.64,186,9,True,4,10.0,17.000000,1,False,45.000000,5720000
2,2,2015,4,55.697556,37.924061,2.74,320,17,True,5,8.0,19.000000,1,False,37.799999,6000000
3,3,1997,4,55.834301,37.352211,2.64,473,17,True,10,9.2,30.799999,2,False,55.000000,8750000
4,4,1974,4,55.815945,37.701008,2.64,143,9,True,7,7.0,33.000000,2,False,48.000000,8500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105107,105107,1984,4,55.761795,37.410034,2.64,474,17,True,13,10.0,33.000000,2,False,57.700001,19300000
105108,105108,2004,4,55.853230,37.646873,2.74,513,17,True,9,7.0,18.000000,1,False,38.000000,9200000
105109,105109,1969,4,55.626068,37.608238,2.50,282,9,True,3,6.7,29.000000,2,False,45.000000,11300000
105110,105110,2008,4,55.872646,37.634228,2.74,128,17,True,9,12.9,33.900002,2,False,64.000000,14800000


In [11]:
df.to_csv('../model_improvement/initial_data.csv', index=False)  # Без индексов

In [ ]:
df["area_per_room"] = df["total_area"] / df["rooms"]
features_filtered = df.drop(["rooms"], axis=1)
features_filtered["area_per_room"] = df["area_per_room"]

In [ ]:
num_cols = ['price','area_per_room','living_area','kitchen_area','build_year','flats_count','floors_total'
           , 'total_area', 'ceiling_height','latitude','longitude']

correlations_with_price = features_filtered[num_cols].corr()

# Построение тепловой карты
plt.figure(figsize=(10, 8))
sns.heatmap(correlations_with_price, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Корреляция между признаками')